# Extension field Paramenters for Altbn

I derive here the parameters for the altbn128 / bn254 pairing friendly curve to efficiently work in [arkworks-rs](https://github.com/arkworks-rs)

Fortunately, there are many commonalities between the BLS12-381 and BN254 curves, and there is already an implementation of the bls12-381 in [ark-test-curves]().


In [30]:
import random

def PairingFriendlyCurve(p: int, G1_params: (int, int), G1_generator: (int, int), u0: int) -> dict:
        assert p.is_prime()
        Fp = GF(p)
        G1 = EllipticCurve(Fp, G1_params)
        r = G1.order()
        G1_gen = G1(*G1_generator)
        _.<x> = Fp[]
        Fp2.<u> = Fp.extension(x^2 + 1)
        nonresidue = u + u0
        _.<t> = Fp2[]
        Fp6.<v> = Fp2.extension(t^3 - nonresidue)
        return {
            "p": p,
            "Fp": GF(p),
            "G1": EllipticCurve(GF(p), G1_params),
            "r": r,
            "G1_gen": G1_gen,
            "Fp2": Fp2,
            "nonresidue": nonresidue,
            "Fp6": Fp6,
            "u": u,
            "v": v,
        }

def generate_random_Fp6(curve):
    (fp2, p, u) = (curve["Fp2"], curve["p"], curve["u"])
    a = fp2(random.randint(1, p-1) + random.randint(1, p-1)*u)
    b = fp2(random.randint(1, p-1) + random.randint(1, p-1)*u)
    c = fp2(random.randint(1, p-1) + random.randint(1, p-1)*u)
    return a + b*v + c*v^2


bls_p = 0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab
bls_ab = (0, 4)
bls_G1_gen = (0x04, 0x0a989badd40d6212b33cffc3f3763e9bc760f988c9926b26da9dd85e928483446346b8ed00e1de5d5ea93e354abe706c)
bls_u0 = 1
BLS = PairingFriendlyCurve(bls_p, bls_ab, bls_G1_gen, bls_u0)

bn_p = 21888242871839275222246405745257275088696311157297823662689037894645226208583
bn_ab = (0, 3)
bn_G1_gen = (1, 2)
bn_u0 = 9
BN = PairingFriendlyCurve(bn_p, bn_ab, bn_G1_gen, bn_u0)

## Extension fields

### Frobenius Coeffs
For example the extensions field tower over $F_{p^2}$, $F_{p^6}$, and $F_{p^12}$. 

This parameters are precomputed to make the extensions faster and not recompute multiple times constants raised to large powers.

#### BLS12-381

In [34]:
# values taken form ark-test-curves
(bls_fp2, bls_fp6, u, v, p) = (BLS["Fp2"], BLS["Fp6"], BLS["u"], BLS["v"], BLS["p"])
should_bls_fp6_coeff_c1 = [
    bls_fp2(1),
    bls_fp2(4002409555221667392624310435006688643935503118305586438271171395842971157480381377015405980053539358417135540939436 * u),
    bls_fp2(793479390729215512621379701633421447060886740281060493010456487427281649075476305620758731620350),
    bls_fp2(u),
    bls_fp2(4002409555221667392624310435006688643935503118305586438271171395842971157480381377015405980053539358417135540939436),
    bls_fp2(793479390729215512621379701633421447060886740281060493010456487427281649075476305620758731620350 * u),
]
should_bls_fp6_coeff_c2 = [
    bls_fp2(1),
    bls_fp2(4002409555221667392624310435006688643935503118305586438271171395842971157480381377015405980053539358417135540939437),
    bls_fp2(4002409555221667392624310435006688643935503118305586438271171395842971157480381377015405980053539358417135540939436),
    bls_fp2(-1),
    bls_fp2(793479390729215512621379701633421447060886740281060493010456487427281649075476305620758731620350),
    bls_fp2(793479390729215512621379701633421447060886740281060493010456487427281649075476305620758731620351),
]
should_bls_fp12_coeff_c1 = [
    bls_fp2(1),
    bls_fp2(
        3850754370037169011952147076051364057158807420970682438676050522613628423219637725072182697113062777891589506424760 + 
        151655185184498381465642749684540099398075398968325446656007613510403227271200139370504932015952886146304766135027 * u
    ),
    bls_fp2(793479390729215512621379701633421447060886740281060493010456487427281649075476305620758731620351),
    bls_fp2(
        2973677408986561043442465346520108879172042883009249989176415018091420807192182638567116318576472649347015917690530 + 
        1028732146235106349975324479215795277384839936929757896155643118032610843298655225875571310552543014690878354869257 * u
    ),
    bls_fp2(793479390729215512621379701633421447060886740281060493010456487427281649075476305620758731620350),
    bls_fp2(
        3125332594171059424908108096204648978570118281977575435832422631601824034463382777937621250592425535493320683825557 +
        877076961050607968509681729531255177986764537961432449499635504522207616027455086505066378536590128544573588734230 * u
    ),
    bls_fp2(-1),
    bls_fp2(
        151655185184498381465642749684540099398075398968325446656007613510403227271200139370504932015952886146304766135027 +
        3850754370037169011952147076051364057158807420970682438676050522613628423219637725072182697113062777891589506424760 * u
    ),
    bls_fp2(4002409555221667392624310435006688643935503118305586438271171395842971157480381377015405980053539358417135540939436),
    bls_fp2(
        1028732146235106349975324479215795277384839936929757896155643118032610843298655225875571310552543014690878354869257 +
        2973677408986561043442465346520108879172042883009249989176415018091420807192182638567116318576472649347015917690530 * u
    ),
    bls_fp2(4002409555221667392624310435006688643935503118305586438271171395842971157480381377015405980053539358417135540939437),
    bls_fp2(
        877076961050607968509681729531255177986764537961432449499635504522207616027455086505066378536590128544573588734230 +
        3125332594171059424908108096204648978570118281977575435832422631601824034463382777937621250592425535493320683825557 * u
    ),
]

def calculate_residue_c1(curve, extension_degree):
    denominator = extension_degree // 2
    residues = []
    for i in range(extension_degree):
        c1 = curve["nonresidue"]^(((curve["p"]^i) - 1) // denominator)
        residues.append(c1)
    return residues

def calculate_residue_c2(curve, extension_degree):
    denominator = extension_degree // 2
    residues = []
    for i in range(extension_degree):
        c2 = curve["nonresidue"]^((2*curve["p"]^i - 2) // 3)
        residues.append(c2)
    return residues

assert calculate_residue_c1(BLS, 6) == should_bls_fp6_coeff_c1
assert calculate_residue_c2(BLS, 6) == should_bls_fp6_coeff_c2
assert calculate_residue_c1(BLS, 12) == should_bls_fp12_coeff_c1

def endomorphism(point, coeffs):
    point_coeffs = [y for x in point.list() for y in x.list()]
    assert len(point_coeffs) == 6 and len(point_coeffs) == len(coeffs)
    prod = [x * y for (x, y) in zip(point_coeffs, coeffs)]
    return prod[0] + prod[1] + (prod[2] + prod[3]) * v + (prod[4] + prod[5]) * v^2

# ---- this test do not work --------
# hardcoded rand element for easyness of debugging, to replace when it works
# random_element_bls_Fp6 = generate_random_Fp6(BLS)
random_element_bls_Fp6 = bls_fp6((7 + 3*u) + (5 + u)*v + (4 + 2*u)*v^2)

manual_endo = endomorphism(random_element_bls_Fp6, should_bls_fp6_coeff_c1)
print(manual_endo)
print(bls_fp6.frobenius_endomorphism()(random_element_bls_Fp6))
assert random_element_bls_Fp6 ^ p == bls_fp6.frobenius_endomorphism()(random_element_bls_Fp6)
# assert manual_endo == random_element_bls_Fp6 ^ p

(1586958781458431025242759403266842894121773480562120986020912974854563298150952611241517463240700*u + 4002409555221667390243872262819042106071364013405322097088511174999789678449011914733561032827110441554859346078383)*v^2 + (u + 3967396953646077563106898508167107235304433701405302465052282437136408245377381528103793658101750)*v + 4002409555221667391037351653548257618692743715038743544149397915280850171459468402160842681902586747175618077698734*u + 7
(1586958781458431025242759403266842894121773480562120986020912974854563298150952611241517463240700*u + 4002409555221667390243872262819042106071364013405322097088511174999789678449011914733561032827110441554859346078387)*v^2 + (4002409555221667389450392872089826593449984311771900650027624434718729185438555427306279383751634135934100614458032*u + 4002409555221667392624310435006688643935503118305586438271171395842971157480381377015405980053539358417135540939436)*v + 4002409555221667393417789825735904156556882819939007885332058136124031650490

#### BN254

In [56]:
fp6_c1 = calculate_residue_c1(BN,6)
fp6_c2 = calculate_residue_c2(BN,6)
fp12_c1 = calculate_residue_c1(BN,12)

## Test that the calculated values are the correct ones
(bn_fp2, bn_fp6, u, v, p)  = (BN["Fp2"], BN["Fp6"], BN["u"], BN["v"], BN["p"])

random_element_Fp6 = bn_fp6((7 + 3*u) + (5 + u)*v + (4 + 2*u)*v^2)

manual_endo = endomorphism(random_element_Fp6, fp6_c1)
print(manual_endo)
print(bn_fp6.frobenius_endomorphism()(random_element_Fp6))
# assert random_element_bls_Fp6 ^ p == bls_fp6.frobenius_endomorphism()(random_element_bls_Fp6)
# assert manual_endo == random_element_bls_Fp6 ^ p


(18688091559996640667624840446475962059012024248151051358417163804016812971406*u + 14969799575116432101891020264074944275207112706867693873992239145150792488569)*v^2 + (2236595495967245188281701248203181795121068902605861227855261137820944008926*u + 3772000881919853765413892760973248631916054879688385666901985624915391085426)*v + 9034561915781853878210447076191061703511379234551486604186546669802400933279*u + 20949905171163978586702161317777928120414510508446003156259765814218852664624
(8922476582650747614709889213007936512066297929813933323450773688530719325695*u + 6426673577740782872568297311219346209443321411639038534719894189783918290246)*v^2 + (8074301469248430268116691354392460132380795221437143288224322511597717333926*u + 8743705428081548640911876894683867316932895715863402578243074720771919807344)*v + 21888242871839275222246405745257275088696311157297823662689037894645226208580*u + 7


In [57]:
fp12_c1

[1,
 16469823323077808223889137241176536799009286646108169935659301613961712198316*u + 8376118865763821496583973867626364092589906065868298776909617916018768340080,
 21888242871839275220042445260109153167277707414472061641714758635765020556617,
 303847389135065887422783454877609941456349188919719272345083954437860409601*u + 11697423496358154304825782922584725312912383441159505038794027105778954184319,
 21888242871839275220042445260109153167277707414472061641714758635765020556616,
 5722266937896532885780051958958348231143373700109372999374820235121374419868*u + 3321304630594332808241809054958361220322477375291206261884409189760185844239,
 21888242871839275222246405745257275088696311157297823662689037894645226208582,
 5418419548761466998357268504080738289687024511189653727029736280683514010267*u + 13512124006075453725662431877630910996106405091429524885779419978626457868503,
 2203960485148121921418603742825762020974279258880205651966,
 2158439548270420933482362229037966514723996196837810

True